In [1]:
import sqlite3
import threading

# create database with queue table
conn = sqlite3.connect('queue_db.sqlite3')
c = conn.cursor()
c.execute('''CREATE TABLE queue
             (id INTEGER PRIMARY KEY AUTOINCREMENT, item TEXT NOT NULL)''')
conn.commit()

# producer
# add item to queue
def add_item(item):
    with conn:
        c.execute("INSERT INTO queue (item) VALUES (?)", (item,))

# consumer
# remove item from queue
def remove_item():
    with conn:
        c.execute("SELECT * FROM queue ORDER BY id LIMIT 1")
        res = c.fetchone()
        if res:
            c.execute("DELETE FROM queue WHERE id=?", (res[0],))
            return res[1]

# manager
# monitor queue
def monitor_queue():
    while True:
        with conn:
            c.execute("SELECT * FROM queue")
            res = c.fetchall()
            print(res)
        time.sleep(1)

# create threads
add_thread = threading.Thread(target=add_item, args=("test item",))
rm_thread = threading.Thread(target=remove_item)
mon_thread = threading.Thread(target=monitor_queue)

# start threads
add_thread.start()
rm_thread.start()
mon_thread.start()

Exception in thread Thread-11:
Traceback (most recent call last):
  File "<ipython-input-1-a17134f78578>", line 15, in add_item
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140651443472256 and this is thread id 140651068389120.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread     self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-a17134f78578>", line 15, in add_item
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140651443472256 and this is thread id 140651068389120.
Thread-12:
Traceback (most recent call last):
  File "<ipython-input-1-a17134f78578>", line 21, in remove_item
sq

In [ ]:
pip install sqlite3

import sqlite3

# Connect to the database
conn = sqlite3.connect('queue.db')

# Create a cursor
cursor = conn.cursor()

# Create the queue table
cursor.execute('''
CREATE TABLE queue (
    id INTEGER PRIMARY KEY,
    item TEXT,
    status TEXT
)
''')

# Commit the changes
conn.commit()

# Close the connection
conn.close()


def producer(item):
    # Connect to the database
    conn = sqlite3.connect('queue.db')

    # Create a cursor
    cursor = conn.cursor()

    # Insert the item into the queue
    cursor.execute('''
    INSERT INTO queue (item, status)
    VALUES (?, ?)
    ''', (item, 'pending'))

    # Commit the changes
    conn.commit()

    # Close the connection
    conn.close()


def consumer():
    # Connect to the database
    conn = sqlite3.connect('queue.db')

    # Create a cursor
    cursor = conn.cursor()

    # Select the first pending item from the queue
    cursor.execute('''
    SELECT id, item FROM queue
    WHERE status = 'pending'
    LIMIT 1
    ''')

    # Fetch the item
    item = cursor.fetchone()

    if item is not None:
        # Process the item
        print('Processing item:', item[1])

        # Update the item's status to "completed"
        cursor.execute('''
        UPDATE queue
        SET status = 'completed'
        WHERE id = ?
        ''', (item[0],))

        # Commit the changes
        conn.commit()

    # Close the connection
    conn.close()



def manager():
    # Check the queue for pending items
    consumer()

    # Add a new item to the queue
    producer('new item')



In [ ]:
import asyncio

async def producer(item):
    # Connect to the database
    conn = sqlite3.connect('queue.db')

    # Create a cursor
    cursor = conn.cursor()

    # Insert the item into the queue
    cursor.execute('''
    INSERT INTO queue (item, status)
    VALUES (?, ?)
    ''', (item, 'pending'))

    # Commit the changes
    conn.commit()

    # Close the connection
    conn.close()

async def consumer():
    # Connect to the database
    conn = sqlite3.connect('queue.db')

    # Create a cursor
    cursor = conn.cursor()

    # Select the first pending item from the queue
    cursor.execute('''
    SELECT id, item FROM queue
    WHERE status = 'pending'
    LIMIT 1
    ''')

    # Fetch the item
    item = cursor.fetchone()

    if item is not None:
        # Process the item
        print('Processing item:', item[1])

        # Update the item's status to "completed"
        cursor.execute('''
        UPDATE queue
        SET status = 'completed'
        WHERE id = ?
        ''', (item[0],))

        # Commit the changes
        conn.commit()

    # Close the connection
    conn.close()

import threading

# Create a thread for the producer function
producer_thread = threading.Thread(target=producer, args=('new item',))

# Create a thread for the consumer function
consumer_thread = threading.Thread(target=consumer)


import asyncio

async def manager():
    # Run the consumer function asynchronously in a separate event loop
    await asyncio.gather(consumer())

    # Add a new item to the queue
    producer('new item')

# Start the event loop
asyncio.run(manager())


